Some notes i'm taking for myself on LFADS controller inputs and feedback timing

In [2]:
#loading data
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '/home/pmalonis/lfads_analysis/src')
import importlib
import segment_submovements as ss
import timing_analysis as ta
import kinematics_and_input as ki
import utils
import pandas as pd
import h5py
from scipy import io
importlib.reload(utils)
importlib.reload(ss)
importlib.reload(ta)
importlib.reload(ki)

trial_type = 'all'

lfads_filename = "/home/pmalonis/lfads_analysis/data/model_output/rockstar_8QTVEk_%s.h5"%trial_type
data_filename = "/home/pmalonis/lfads_analysis/data/intermediate/rockstar.p"
inputInfo_filename = "/home/pmalonis/lfads_analysis/data/model_output/rockstar_inputInfo.mat"

df = pd.read_pickle(data_filename)
input_info = io.loadmat(inputInfo_filename)
with h5py.File(lfads_filename) as h5file:
    co = h5file['controller_outputs'].value
    dt = utils.get_dt(h5file, input_info)
    
used_inds = utils.get_indices(input_info, trial_type)

In [2]:
#plotting first trial
importlib.reload(ss)
importlib.reload(ta)
ss.plot_trial(df.loc[0], co[0, :,:], dt)
ss.plot_trial(df.loc[1], co[1, :,:], dt)


In [6]:
import seaborn as sns
sns.jointplot(co[:200,::20,0].flatten(), co[:200,::20,1].flatten(), kind='scatter')

In [7]:
targets = df.kinematic.query('hit_target')
post_target = []
interval = 0.2 # interval to leave out
trial_len = co.shape[1]*dt
for i in range(targets.index[-1][0] + 1):
    t_start = np.round(targets.loc[i]['hit_target'].index.values/dt).astype(int)
    t_stop = t_start + int(interval/dt)
    idx = np.concatenate([np.arange(stop, start, dtype=int) for start, stop in zip(t_start[1:], t_stop[:-1]) if start < trial_len/dt])
    post_target.append(idx)

In [8]:
input0 = np.concatenate([co[i,post_target[i],0] for i in range(200)])
input1 = np.concatenate([co[i,post_target[i],1] for i in range(200)])
sns.jointplot(input0[::20], input1[::20])

In [9]:
import importlib
import segment_submovements as ss
importlib.reload(ss)

trial_df = df.loc[0]
ss.add_target_position_column(trial_df)

In [10]:
import importlib
import segment_submovements as ss
importlib.reload(ss)

trial_df = df.loc[0]
#ss.add_target_position_column(trial_df)
targets = trial_df.kinematic.query('hit_target')
idx_target = np.where(trial_df.kinematic['hit_target'].values)[0]
target_x = np.zeros(trial_df.shape[0])
target_y = np.zeros(trial_df.shape[0])
for i in range(targets.shape[0]-1):
    target_x[idx_target[i]:idx_target[i+1]] = targets['x'].iloc[i]
    target_y[idx_target[i]:idx_target[i+1]] = targets['y'].iloc[i]

trial_df['target_x'] = target_x
trial_df['target_y'] = target_y

In [3]:
importlib.reload(ss)
trial_idx = 0
trial_df = df.loc[trial_idx]
c=ss.peak_error(trial_df)
plt.figure(figsize=(10,5))
plt.plot(c[:3860]/2,'r')
plt.plot(np.arange(0, 3860, 10), co[trial_idx,:,0])
plt.plot(np.arange(0, 3860, 10), co[trial_idx,:,1])
plt.vlines(trial_df.loc[:3.86].kinematic.query('hit_target').index.values*1000, -1, 1)

In [ ]:
importlib.reload(ss)
trial_idx = 8
trial_df = df.loc[trial_idx]
c=ss.peak_error(trial_df)
plt.figure(figsize=(10,5))
plt.plot(c[:3860]/2,'r')
plt.plot(np.arange(0, 3860, 10), co[trial_idx,:,0])
plt.plot(np.arange(0, 3860, 10), co[trial_idx,:,1])
plt.vlines(trial_df.loc[:3.86].kinematic.query('hit_target').index.values*1000, -1, 1)

ss.plot_submovements(trial_df)

In [13]:
importlib.reload(ta)
peaks = ta.get_peaks(co[:,:,:1], dt, min_height=[.4], exclude_post_target=.2, df=df)

In [3]:
error = []
peaks = peaks.flatten()
for i in range(len(peaks)):
    trial_df = df.loc[i]
    c = ss.peak_error(df.loc[i])
    for peak in peaks[i]:
        c = ss.peak_error(trial_df.loc[peak-.1:peak+.1])
        error.append(np.mean(c))

import utils
utils.polar_hist(error, 12)

In [10]:
utils.polar_hist(np.array(error)%(2*np.pi), 12)

In [ ]:
for j in range(1):
    controls = [np.array([np.random.rand() * 3.86]) for i in range(307)]
    error = []
    for i in range(len(controls)):
        trial_df = df.loc[i]
        c = ss.peak_error(df.loc[i])
        for control in controls[i]:
            c = ss.peak_error(trial_df.loc[control-.1:control+.1])
            error.append(np.mean(c))

    utils.polar_hist(np.array(error)%(2*np.pi), 12)

In [14]:
importlib.reload(ki)

trial_idx = 0
dt = 0.01



In [20]:
importlib.reload(ki)
from IPython.core.display import display, HTML
i=6
a,f = ki.animate_trial_pandas(df.loc[i],co[i,:,:].T,.01)
HTML(a.to_jshtml())

In [4]:
import sys
sys.path.insert(0, '/home/pmalonis/lfads_analysis/src')
from power_law import get_curvature

n_trials = df.index[-1][0] + 1
K = []
trial_len = co.shape[1] * dt
for i in range(10):
    trial_df = df.loc[i]
    t_target = trial_df.kinematic.query('hit_target').index
    idx = range(trial_df.shape[0])
    t = trial_df.index[idx]
    x = trial_df.kinematic['x'].iloc[idx].values
    y = trial_df.kinematic['y'].iloc[idx].values
    idx = [trial_df.index.get_loc(t, 'nearest') for t in np.arange(0, trial_len, dt)]
    K.append(get_curvature(x,y,t)[idx]
    
curv = np.concatenate(K)
np.corrcoef(curv, co[:10,:,0].flatten())[0,1]

In [5]:
for i in range(10):
    fig, ax = plt.subplots(figsize=(15,5))
    ax.plot(K[i])
    ax.set_ylim([0, 5])
    ax2 = ax.twinx()
    ax2.plot(co[i,:,0],'r')
    t_targets = df.loc[i].loc[:trial_len].kinematic.query('hit_target').index
    plt.vlines(t_targets/dt,0, np.max(co[i,:,0]))

In [1]:
import segment_submovements as ss
importlib.reload(ss)

for i in range(10): 
    ss.plot_trajectory_co(df.loc[i], co[i,:,0], dt)
    

In [53]:
from numpy.linalg import norm
ntrials = co.shape[0]
m=co[:,:,0].mean()
nwin = 12
K_trial = K[0]
straigt = np.concatenate([[norm(co[0,i-nwin:i+nwin,0]-m) for i in range(nwin,ntrials-nwin) if np.all(K_trial[i-nwin:i+nwin]<.1)] for K_trial in K[:10]])
curve = np.concatenate([[norm(co[0,i-nwin:i+nwin,0]-m) for i in range(nwin,ntrials-nwin) if np.any(K_trial[i-nwin:i+nwin]>.2)] for K_trial in K[:10]])

In [54]:
straight = straigt[~np.isnan(straigt)]
print(np.mean(straight))
print(np.mean(curve))

from scipy.stats import wilcoxon, mannwhitneyu
mannwhitneyu(straight, curve)

In [58]:
from seaborn import barplot

barplot([1, 2], [np.mean(straight), np.mean(curve)])